# Diagnostic : Eddy heat flux 

We show here the usage of the eddy heat flux diagnostics with is a simple 2-layer channel QG atmosphere truncated at wavenumber 2 on a beta-plane with a simple orography (a montain and a valley). 

## Modules import

First, setting the path and loading of some modules

In [ ]:
import sys, os

In [ ]:
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','sans-serif':['Times'],'size':14})

Initializing the random number generator (for reproducibility). -- Disable if needed.

In [ ]:
np.random.seed(210217)

Importing the model's modules

In [ ]:
from qgs.params.params import QgParams
from qgs.integrators.integrator import RungeKuttaIntegrator, RungeKuttaTglsIntegrator
from qgs.functions.tendencies import create_tendencies
from qgs.plotting.util import std_plot

## Systems definition

General parameters

In [ ]:
# Time parameters
dt = 0.1
# Saving the model state n steps
write_steps = 5

number_of_trajectories = 1
number_of_perturbed_trajectories = 10

Setting some model parameters

In [ ]:
# Model parameters instantiation with some non-default specs
model_parameters = QgParams({'phi0_npi': np.deg2rad(50.)/np.pi, 'hd':0.045})
# Mode truncation at the wavenumber 2 in both x and y spatial coordinate
model_parameters.set_atmospheric_channel_fourier_modes(2, 2)

# Changing (increasing) the orography depth and the meridional temperature gradient
model_parameters.ground_params.set_orography(0.2, 1)
model_parameters.atemperature_params.set_thetas(0.1, 0)

In [ ]:
# Printing the model's parameters
model_parameters.print_params()

Creating the tendencies function

In [ ]:
%%time
f, Df = create_tendencies(model_parameters)

## Time integration

Defining an integrator

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

Start on a random initial condition and integrate over a transient time to obtain an initial condition on the attractors

In [ ]:
%%time
ic = np.random.rand(model_parameters.ndim)*0.1
integrator.integrate(0., 200000., dt, ic=ic, write_steps=0)
time, ic = integrator.get_trajectories()

Now integrate to obtain a trajectory on the attractor

In [ ]:
%%time
integrator.integrate(0., 100000., dt, ic=ic, write_steps=write_steps)
reference_time, reference_traj = integrator.get_trajectories()

In [ ]:
varx = 0
vary = 1
varz = 2

fig = plt.figure(figsize=(10, 8))
axi = fig.add_subplot(111, projection='3d')

axi.scatter(reference_traj[varx], reference_traj[vary], reference_traj[varz], s=0.2);

axi.set_xlabel('$'+model_parameters.latex_var_string[varx]+'$')
axi.set_ylabel('$'+model_parameters.latex_var_string[vary]+'$')
axi.set_zlabel('$'+model_parameters.latex_var_string[varz]+'$');

In [ ]:
varx = 2
vary = 1
plt.figure(figsize=(10, 8))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
var = 1
plt.figure(figsize=(10, 8))

plt.plot(model_parameters.dimensional_time*reference_time, reference_traj[var])

plt.xlabel('time (days)')
plt.ylabel('$'+model_parameters.latex_var_string[var]+'$');

## Showing the eddy heat flux


Creating the diagnostics:

* Wind, temperature and eddy heat flux at the middle of the atmosphere:

In [ ]:
from qgs.diagnostics.wind import MiddleAtmosphericVWindDiagnostic, MiddleAtmosphericWindIntensityDiagnostic
from qgs.diagnostics.temperatures import MiddleAtmosphericTemperatureDiagnostic
from qgs.diagnostics.eddy import MiddleAtmosphericEddyHeatFluxDiagnostic, MiddleAtmosphericEddyHeatFluxProfileDiagnostic

In [ ]:
vwind = MiddleAtmosphericVWindDiagnostic(model_parameters)
temp = MiddleAtmosphericTemperatureDiagnostic(model_parameters)
wind = MiddleAtmosphericWindIntensityDiagnostic(model_parameters)

In [ ]:
eddy = MiddleAtmosphericEddyHeatFluxDiagnostic(model_params=model_parameters, delta_x=0.25, delta_y=0.15)
eddyp = MiddleAtmosphericEddyHeatFluxProfileDiagnostic(model_params=model_parameters, delta_x=0.25, delta_y=0.15)

* For the nondimensional variables $\psi_2$ and $\psi_3$:

In [ ]:
from qgs.diagnostics.variables import VariablesDiagnostic
from qgs.diagnostics.multi import MultiDiagnostic

In [ ]:
variable_nondim = VariablesDiagnostic([2, 1], model_parameters, False)

In [ ]:
# setting also the background
background = VariablesDiagnostic([2, 1], model_parameters, False)
background.set_data(reference_time, reference_traj)

Creating a multi diagnostic with all the diagnostics:

In [ ]:
m = MultiDiagnostic(2,2)
m.add_diagnostic(variable_nondim,
                 diagnostic_kwargs={'show_time': False, 'background': background},
                 plot_kwargs={'ms': 0.2})

# m.add_diagnostic(vwind)#,
                 #diagnostic_kwargs={'style': 'contour', 'contour_labels': False},
                 #plot_kwargs={'colors': 'k'})
        
m.add_diagnostic(eddy, plot_kwargs={'cmap':plt.get_cmap('magma')})
m.add_diagnostic(eddyp, plot_kwargs={'figsize':(10,8)})

m.add_diagnostic(temp)
                 
m.set_data(reference_time, reference_traj)

and show an interactive animation:

In [ ]:
rc('font',**{'family':'serif','sans-serif':['Times'],'size':12})
m.animate(figsize=(23,12))

or a movie:

In [ ]:
rc('font',**{'family': 'serif','sans-serif': ['Times'],'size': 12})
m.movie(figsize=(23,10), anim_kwargs={'interval': 100, 'frames': 1000})